In [3]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob
from sklearn.metrics import classification_report
from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt

#Data preprocessing to removes @usernames,urls,symbols and makes all text lowercase
def preprocess_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    text = re.sub('@[^\s]+','', text)
    text = text.lower().replace("ё", "е")
    text = re.sub('[^a-zA-Zа-яА-Я]+',' ', text)
    text = re.sub(' +',' ', text)
    return text.strip()


#from google.colab import files   
#uploaded = files.upload()
#nltk.download('punkt')

#Global variable
batchsize = 100000 

In [4]:
#load data from csv/excel/json
#data=pd.read_csv('textblob_sample_5k.csv',encoding = 'unicode_escape')
#data=pd.read_csv('textblob_sentiment_1M.csv',encoding = "ISO-8859–1")
data = pd.read_csv('vote_all_no_conflict.csv')

data.head()

,Tweet ID,text,SentiStrength,Vader,Textblob,Vote
0,1278320583718178816,Business Group Complains Trump H-1B Reform Boo...,Negative,Negative,Negative,Negative
1,1278346691171586049,That moron trump vows to veto the Defense Bill...,Neutral,Negative,Negative,Negative
2,1278368975689220097,@JoeBiden Debate President Trump. PROVE you do...,Neutral,Neutral,Neutral,Neutral
3,1278368976960184320,@PamelaStovall6 @ChuckGrassley @realDonaldTrum...,Neutral,Positive,Neutral,Neutral
4,1278368971314597890,@Jorgensen4POTUS @RealSpikeCohen Just found ou...,Positive,Positive,Positive,Positive


In [5]:
#data["processed_text"] =  data['full_text'].apply(preprocess_text)
data["processed_text"] = data['text'].apply(preprocess_text)

In [6]:
pd.options.display.max_colwidth = 150
data.head()


,Tweet ID,text,SentiStrength,Vader,Textblob,Vote,processed_text
0,1278320583718178816,Business Group Complains Trump H-1B Reform Boosting U.S. Graduates. Big tech is whining that they will have to hire American instead of cheap fore...,Negative,Negative,Negative,Negative,business group complains trump h b reform boosting u s graduates big tech is whining that they will have to hire american instead of cheap foreign...
1,1278346691171586049,"That moron trump vows to veto the Defense Bill if it includes renaming bases. So once again, military salaries and defense preparedness are second...",Neutral,Negative,Negative,Negative,that moron trump vows to veto the defense bill if it includes renaming bases so once again military salaries and defense preparedness are second i...
2,1278368975689220097,@JoeBiden Debate President Trump. PROVE you don’t have dementia. #DementiaJoeCantDebate #JoeBidenScaredToDebate,Neutral,Neutral,Neutral,Neutral,debate president trump prove you don t have dementia dementiajoecantdebate joebidenscaredtodebate
3,1278368976960184320,@PamelaStovall6 @ChuckGrassley @realDonaldTrump Democrats are always saving @GOP dinosaurs like grassley after they fvck up,Neutral,Positive,Neutral,Neutral,democrats are always saving dinosaurs like grassley after they fvck up
4,1278368971314597890,@Jorgensen4POTUS @RealSpikeCohen Just found out about you and so far I love your policies and what you have to say. Ofc I still need to read more ...,Positive,Positive,Positive,Positive,just found out about you and so far i love your policies and what you have to say ofc i still need to read more about your stances and plans but i...


In [7]:
#Converting the text into tokens and getting the counts of each token based on the ngrams specified. 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
#cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = nltk.word_tokenize)
#cv=CountVectorizer(stop_words='english')
text_counts = cv.fit_transform(data['processed_text'])

In [8]:
text_counts
#cv.vocabulary_
# Import LabelEncoder
#from sklearn import preprocessing
#creating labelEncoder
#le = preprocessing.LabelEncoder()
#label = le.fit_transform(data['Sentiment'])

<2475775x247849 sparse matrix of type '<class 'numpy.int64'>'
	with 25453762 stored elements in Compressed Sparse Row format>

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

X_train, X_test, Y_train, Y_test = train_test_split(text_counts,data['Vote'], test_size=0.25, random_state=5)

In [10]:
def batch_xy(input, output, batchsize):
  for i in range(0, input.shape[0], batchsize):
    yield input[i:i + batchsize, :], output[i:i + batchsize]

def batch_x(input, batchsize):
  for i in range(0, input.shape[0], batchsize):
    yield input[i:i + batchsize, :]

def report(name, target, pred):
  label = "[" + name + "] classification report:"
  print(label)
  print(classification_report(target, pred)) 

def roc(name, model, input, target):
  label = "[" + name + "] roc curve:"
  print(label)
  plt.figure()
  roc_plt = plot_roc_curve(model, input, target)
  plt.show()


In [13]:

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier


clf=RandomForestClassifier(n_estimators=100,criterion='gini',warm_start=True)
#for index in range(10):

for x, y in batch_xy(X_train, Y_train, batchsize):
  clf.fit(x, y)
  
y_pred = np.array([])
for x in batch_x(X_test, batchsize):
  y_pred = np.append(y_pred, clf.predict(x))

accuracy_score_clf = metrics.accuracy_score(y_pred, Y_test)
print('accuracy_score_RandomForest = '+str('{:4.2f}'.format(accuracy_score_clf*100))+'%')
report("Initial RF with text counts from Count Vectorizer", Y_test, y_pred)

#roc("Initial MNB with text counts from Count Vectorizer", MNB, X_test, Y_test)

/Users/as76643/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
/Users/as76643/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
/Users/as76643/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
/Users/as76643/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
/Users/as76643/anaconda3

accuracy_score_RandomForest = 81.97%
[Initial RF with text counts from Count Vectorizer] classification report:
              precision    recall  f1-score   support

    Negative       0.88      0.76      0.81    192007
     Neutral       0.76      0.95      0.85    261925
    Positive       0.88      0.69      0.78    165012

    accuracy                           0.82    618944
   macro avg       0.84      0.80      0.81    618944
weighted avg       0.83      0.82      0.82    618944



In [16]:
#searching for optimum number of ensembles
clf=RandomForestClassifier(criterion='gini',warm_start=True)
parameters = {'n_estimators':[50,100,500,1000]}
search =  GridSearchCV(clf,parameters)
search.fit(X_train[0:0 + batchsize, :],Y_train[0:0 + batchsize]) #Only run through batchsize to determine the best params (whole dataset takes too much resource)
bestparams =search.best_params_
bestparams

{'n_estimators': 1000}

In [17]:
clf_best = RandomForestClassifier(n_estimators=bestparams['n_estimators'],criterion='gini',warm_start=True)

for x, y in batch_xy(X_train, Y_train, batchsize):
  clf_best.fit(x, y)
  
y_pred = np.array([])
for x in batch_x(X_test, batchsize):
  y_pred = np.append(y_pred, clf_best.predict(x))

accuracy_score_clf = metrics.accuracy_score(y_pred, Y_test)
print('accuracy_score_RF = '+str('{:4.2f}'.format(accuracy_score_clf*100))+'%')
report("Best Params RF with text counts from Count Vectorizer", Y_test, y_pred)


/Users/as76643/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
/Users/as76643/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
/Users/as76643/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
/Users/as76643/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:359: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
/Users/as76643/anaconda3

accuracy_score_RF = 82.59%
[Best Params RF with text counts from Count Vectorizer] classification report:
              precision    recall  f1-score   support

    Negative       0.89      0.76      0.82    192007
     Neutral       0.77      0.95      0.85    261925
    Positive       0.89      0.71      0.79    165012

    accuracy                           0.83    618944
   macro avg       0.85      0.81      0.82    618944
weighted avg       0.84      0.83      0.82    618944



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words="english",norm="l2")
text_count_2 = tfidf.fit_transform(data['processed_text'])

#splitting the data in test and training
#x_train, x_test, y_train, y_test = train_test_split(text_count_2, data['Sentiment'],test_size=0.25,random_state=5)
x_train, x_test, y_train, y_test = train_test_split(text_count_2, data['Vote'],test_size=0.25,random_state=5)

#Models - reinitialize models since running fit on already fitted model may return something weird (I could be wrong on this, not sure how sklearn fit works initially)
clf=RandomForestClassifier(n_estimators=1000,criterion='gini',warm_start=True)
#for index in range(10):

for x, y in batch_xy(x_train, y_train, batchsize):
  clf.fit(x, y)
  break
    
y_pred = np.array([])
for x in batch_x(x_test, batchsize):
  y_pred = np.append(y_pred, clf.predict(x))

accuracy_score_clf = metrics.accuracy_score(y_pred, y_test)
print('accuracy_score_RandomForest = '+str('{:4.2f}'.format(accuracy_score_clf*100))+'%')
report("Initial RF with text counts from Count Vectorizer", y_test, y_pred)
#roc("Initial MNB with text counts from Count Vectorizer", MNB, X_test, Y_test)

In [19]:
#searching for best params

#searching for optimum number of ensembles
clf=RandomForestClassifier(criterion='gini',warm_start=True)
parameters = {'n_estimators':[50,100,500,1000]}
search =  GridSearchCV(clf,parameters)
search.fit(X_train[0:0 + batchsize, :],y_train[0:0 + batchsize]) #Only run through batchsize to determine the best params (whole dataset takes too much resource)
bestparams =search.best_params_
print(bestparams)


{'n_estimators': 1000}


In [22]:
clf_best = RandomForestClassifier(n_estimators=bestparams['n_estimators'],criterion='gini',warm_start=True)


for x, y in batch_xy(x_train, y_train, batchsize):
  clf_best.fit(x, y)
  break
  
y_pred = np.array([])
for x in batch_x(x_test, batchsize):
  y_pred = np.append(y_pred, clf_best.predict(x))

accuracy_score_clf = metrics.accuracy_score(y_pred, y_test)
print('accuracy_score_RF = '+str('{:4.2f}'.format(accuracy_score_clf*100))+'%')
report("Best Params RF with text counts from Tfidf Vectorizer", y_test, y_pred)


accuracy_score_RF = 84.37%
[Best Params RF with text counts from Tfidf Vectorizer] classification report:
              precision    recall  f1-score   support

    Negative       0.87      0.81      0.84    192007
     Neutral       0.81      0.94      0.87    261925
    Positive       0.89      0.74      0.81    165012

    accuracy                           0.84    618944
   macro avg       0.86      0.83      0.84    618944
weighted avg       0.85      0.84      0.84    618944



In [ ]:

text_counts.toarray()
# Convert raw frequency counts into TF-IDF (Term Frequency -- Inverse Document Frequency) values
from sklearn.feature_extraction.text import TfidfTransformer
fooTfmer = TfidfTransformer()

# Again, fit and transform
docs_tfidf = fooTfmer.fit_transform(text_counts)

#splitting the data in test and training
#from sklearn.model_selection() import train_test_split()
#x_train, x_test, y_train, y_test = train_test_split(docs_tfidf, data['Sentiment'],test_size=0.25,random_state=5)
x_train_tf, x_test_tf, y_train_tf, y_test_tf = train_test_split(docs_tfidf, data['Vote'],test_size=0.25,random_state=5)

#Models - reinitialize models since running fit on already fitted model may return something weird (I could be wrong on this, not sure how sklearn fit works initially)
clf=RandomForestClassifier(n_estimators=100,criterion='gini',warm_start=True)
#for index in range(10):

for x, y in batch_xy(x_train_tf, y_train_tf, batchsize):
  clf.fit(x, y)
  
y_pred = np.array([])
for x in batch_x(x_test_tf, batchsize):
  y_pred = np.append(y_pred, clf.predict(x))

accuracy_score_clf = metrics.accuracy_score(y_pred, y_test_tf)
print('accuracy_score_RandomForest = '+str('{:4.2f}'.format(accuracy_score_clf*100))+'%')
report("Initial RF with text counts from Count Vectorizer", y_test_tf, y_pred)


In [ ]:
#searching for best params

clf=RandomForestClassifier(criterion='gini',warm_start=True)
parameters = {'n_estimators':[50,100,500,1000]}
search =  GridSearchCV(clf,parameters)
search.fit(x_train_tf[0:0 + batchsize, :],y_train_tf[0:0 + batchsize]) #Only run through batchsize to determine the best params (whole dataset takes too much resource)
bestparams =search.best_params_
print(bestparams)


In [ ]:
clf_best = RandomForestClassifier(n_estimators=bestparams['n_estimators'],criterion='gini',warm_start=True)

for x, y in batch_xy(x_train_tf, y_train_tf, batchsize):
  clf_best.fit(x, y)
  
y_pred = np.array([])
for x in batch_x(x_test_tf, batchsize):
  y_pred = np.append(y_pred, clf_best.predict(x))

accuracy_score_clf = metrics.accuracy_score(y_pred, y_test_tf)
print('accuracy_score_RF = '+str('{:4.2f}'.format(accuracy_score_clf*100))+'%')
report("Best Params RF with text counts from Count Vectorizer", y_test_tf, y_pred)
